# Join Data Bases

@roman

5 Oct 2024

---
# Settings

In [1]:
import os
import pandas as pd
import geopandas as gpd
import uuid
import re

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Settings
# show 100 columns in pandas
pd.set_option('display.max_columns', 100)

In [3]:
# params
column_mapping = {
    "1/2_banos": "half_bathrooms",
    "banos": "full_bathrooms",
    "clase_inmuebles_zona": "property_class_more_common_in_area",
    "neighborhood": "neighborhood",
    "constructor": "constructor",
    "cp": "zip_code",
    "cuenta_predial": "property_account_id",
    "descripcion_instalacion_especial_comunes": "special_common_installations_description",
    "descripcion_instalacion_especial_privativas": "special_private_installations_description",
    "descripcion_obra_complementaria_comun": "complementary_common_works_description",
    "edad_meses": "age_in_months",
    "estacionamiento": "parking_lots",
    "valuation_date": "valuation_date",
    "grado_avance_areas_comunes": "common_areas_completion_percentage",
    "grado_terminacion_obra": "private_works_completion_percentage",
    "id_agua_potable": "potable_water_service_id",
    "id_alumbrado": "lighting_service_id",
    "id_banquetas": "sidewalk_id",
    "id_calidad_proyecto": "project_quality_id",
    "id_clase": "property_class_id",
    "id_conservacion": "conservation_status_id",
    "id_ctl": "controller_id",
    "id_elevador": "elevator_service_id",
    "id_entidad": "state_id",
    "id_equipamiento": "equipment_id",
    "id_guarniciones": "curb_id",
    "id_infraestructura": "infrastructure_service_id",
    "id_material_vialidades": "road_materials_id",
    "municipality_id": "municipality_id",
    "id_otorgante": "grantor_id",
    "id_proximidad_urbana": "urban_proximity_id",
    "id_recoleccion_aguas": "water_collection_service_id",
    "id_suministro_electrico": "electrical_supply_service_id",
    "id_suministro_telefono": "telephone_service_supply_id",
    "id_tipo": "property_type_id",
    "id_transporte_urbano": "public_transportation_service_id",
    "id_uv": "valuation_unit_id",
    "id_vp": "valuation_professional_id",
    "latitudee": "latitudee",
    "latitudeee": "latitudeee",
    "nivel": "level",
    "niveles": "total_levels",
    "proposito": "purpose_of_appraisal",
    "recamaras": "bedrooms",
    "sup_accesoria": "accessory_area",
    "sup_construida": "built_area",
    "sup_vendible": "saleable_area",
    "sup_terreno": "land_area",
    "transporte_urbano_distancia": "distance_to_public_transportation",
    "unidades_rentables": "rentable_units",
    "unidades_rentables_sujeto": "rentable_units_subject_property",
    "uso_actual": "current_use",
    "valor_comparativo": "market_price",
    "valor_concluido": "price",
    "valor_fisico_construccion": "built_price",
    "valor_fisico_terreno": "land_price",
    "vias_acceso": "access_routes",
    "vida_util_remanente": "remaining_useful_life"
}

# folder data
folder_data = "../../data/raw/"
folder_data_interim = "../../data/interim/"

---
# Data

## Read

In [4]:
def clean_column_names(df):
    # Function to clean column names
    def clean_column(col):
        col = col.lower()  # Lowercase all characters
        col = col.strip()  # Trim leading/trailing whitespace
        col = col.replace(' ', '_')  # Replace spaces with underscores
        # remove accents
        col = col.replace('á', 'a')
        col = col.replace('é', 'e')
        col = col.replace('í', 'i')
        col = col.replace('ó', 'o')
        col = col.replace('ú', 'u')
        col = col.replace('ñ', 'n')
        col = col.replace('ü', 'u')
        col = col.replace('ä', 'a')
        col = col.replace('ë', 'e')
        col = col.replace('ï', 'i')
        col = col.replace('ö', 'o')
        col = col.replace('ü', 'u')
        # remove -1 
        col = col.replace('-1', '')
        
        return col

    # Apply the cleaning function to all columns
    df.columns = [clean_column(col) for col in df.columns]
    return df

def read_all_csv_files_in_folder(folder_path):
    # Read all csv files in a folder
    df = pd.DataFrame()
    for file in os.listdir(folder_path):
        if file.endswith('.csv'):
            print(f'Reading {file}')
            df = pd.concat([df, pd.read_csv(os.path.join(folder_path, file))], ignore_index=True)
            print(f'Shape: {df.shape}')
    return df

def generate_uuid_map(series):
    unique_values = series.unique()
    uuid_map = {val: str(uuid.uuid4()) for val in unique_values}
    return uuid_map

In [5]:
# read all csv files in folder
df_properties = clean_column_names(read_all_csv_files_in_folder(folder_data))

Reading ITAM_2022_CSV.csv
Shape: (385251, 58)
Reading ITAM_2023_CSV.csv
Shape: (872455, 58)
Reading ITAM_2021_CSV.csv


/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_87327/3497823692.py:35: DtypeWarning: Columns (8,51,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([df, pd.read_csv(os.path.join(folder_path, file))], ignore_index=True)


Shape: (1363759, 58)
Reading ITAM_2020_CSV.csv


/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_87327/3497823692.py:35: DtypeWarning: Columns (8,51,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([df, pd.read_csv(os.path.join(folder_path, file))], ignore_index=True)


Shape: (1838477, 58)
Reading ITAM_2019_CSV.csv
Shape: (2339244, 58)


In [6]:
# see columns 8, 51 and 56
df_properties.iloc[:, [8, 51, 56]].describe()

,descripcion_instalacion_especial_privativas,uso_actual,vias_acceso
count,319545,1534879,1545584
unique,10725,275290,655115
top,NO APLICA,CASA HABITACION,AL NORTE POR EL LIBRAMIENTO MATAMOROS-MONTERREY.
freq,76276,276178,3959


In [7]:
# see df
print(df_properties.shape[0])
df_properties.head(2)

2339244


,1/2_banos,banos,clase_inmuebles_zona,colonia,constructor,cp,cuenta_predial,descripcion_instalacion_especial_comunes,descripcion_instalacion_especial_privativas,descripcion_obra_complementaria_comun,edad_meses,estacionamiento,fecha_avaluo,grado_avance_areas_comunes,grado_terminacion_obra,id_agua_potable,id_alumbrado,id_banquetas,id_calidad_proyecto,id_clase,id_conservacion,id_ctl,id_elevador,id_entidad,id_equipamiento,id_guarniciones,id_infraestructura,id_material_vialidades,id_municipio,id_otorgante,id_proximidad_urbana,id_recoleccion_aguas,id_suministro_electrico,id_suministro_telefono,id_tipo,id_transporte_urbano,id_uv,id_vp,latitude,longitude,nivel,niveles,proposito,recamaras,sup_accesoria,sup_construida,sup_vendible,sup_terreno,transporte_urbano_distancia,unidades_rentables,unidades_rentables_sujeto,uso_actual,valor_comparativo,valor_concluido,valor_fisico_construccion,valor_fisico_terreno,vias_acceso,vida_util_remanente
0,1,2,4,VISTA HERMOSA HUITEPEC,NO APLICA,29263,NaN,NaN,NaN,NaN,36,1,14/06/2022,NaN,100.0,NaN,NaN,NaN,NaN,4,4,1844221,0.0,7,1,NaN,1,2,78,40044,3,NaN,NaN,NaN,2,NaN,9058,1844680,16.737283,-92.672659,3,3,S/D,3,10.75,216.81,227.56,302.7,NaN,1.0,NaN,NaN,3716737,3717000,2532281.13,1033417.8,NaN,67
1,1,2,4,MAYA,NO APLICA,97134,NaN,NaN,NaN,NaN,204,1,15/11/2022,NaN,100.0,NaN,NaN,NaN,NaN,4,4,1641085,0.0,31,2,NaN,1,2,50,40021,2,NaN,NaN,NaN,2,NaN,9070,305318,21.014297,-89.575189,3,2,S/D,3,13.38,141.55,154.93,198.0,NaN,1.0,NaN,NaN,2348706,2349000,1111845.21,984852.0,NaN,53


## Format

In [8]:
# rename
df_properties = df_properties.rename(columns=column_mapping)

# see df
df_properties.head(2)

,half_bathrooms,full_bathrooms,property_class_more_common_in_area,neighborhood,constructor,zip_code,property_account_id,special_common_installations_description,special_private_installations_description,complementary_common_works_description,age_in_months,parking_lots,valuation_date,common_areas_completion_percentage,private_works_completion_percentage,potable_water_service_id,lighting_service_id,sidewalk_id,project_quality_id,property_class_id,conservation_status_id,controller_id,elevator_service_id,state_id,equipment_id,curb_id,infrastructure_service_id,road_materials_id,municipality_id,grantor_id,urban_proximity_id,water_collection_service_id,electrical_supply_service_id,telephone_service_supply_id,property_type_id,public_transportation_service_id,valuation_unit_id,valuation_professional_id,latitude,longitude,level,total_levels,purpose_of_appraisal,bedrooms,accessory_area,built_area,saleable_area,land_area,distance_to_public_transportation,rentable_units,rentable_units_subject_property,current_use,market_price,price,built_price,land_price,access_routes,remaining_useful_life
0,1,2,4,VISTA HERMOSA HUITEPEC,NO APLICA,29263,NaN,NaN,NaN,NaN,36,1,14/06/2022,NaN,100.0,NaN,NaN,NaN,NaN,4,4,1844221,0.0,7,1,NaN,1,2,78,40044,3,NaN,NaN,NaN,2,NaN,9058,1844680,16.737283,-92.672659,3,3,S/D,3,10.75,216.81,227.56,302.7,NaN,1.0,NaN,NaN,3716737,3717000,2532281.13,1033417.8,NaN,67
1,1,2,4,MAYA,NO APLICA,97134,NaN,NaN,NaN,NaN,204,1,15/11/2022,NaN,100.0,NaN,NaN,NaN,NaN,4,4,1641085,0.0,31,2,NaN,1,2,50,40021,2,NaN,NaN,NaN,2,NaN,9070,305318,21.014297,-89.575189,3,2,S/D,3,13.38,141.55,154.93,198.0,NaN,1.0,NaN,NaN,2348706,2349000,1111845.21,984852.0,NaN,53


In [9]:
# Converting columns to appropriate dtypes
print("datetime columns")
df_properties['valuation_date'] = pd.to_datetime(df_properties['valuation_date'], format='%d/%m/%Y')

# Convert IDs to int64 (if they should not have decimals)
print("id columns")
id_columns = df_properties.columns[df_properties.columns.str.contains('_id')]
# pop valuation ids
valuation_identifiers = ['controller_id', 'grantor_id', 'valuation_unit_id', 'valuation_professional_id']
id_columns = id_columns.drop(valuation_identifiers)
df_properties[valuation_identifiers].isna().sum()
df_properties[id_columns] = df_properties[id_columns].fillna(-1).astype('int64').replace(-1, pd.NA)

# Conver valuation identifiers to hash
print("valuation identifiers")
for col in valuation_identifiers:
    # Generate a mapping from the original values to a new UUID
    col_mapping = generate_uuid_map(df_properties[col])
    # Replace the original values with the UUIDs
    df_properties[col] = df_properties[col].map(col_mapping)

# Convert price columns to float64
print("price columns")
price_columns = df_properties.columns[df_properties.columns.str.contains('price')]
df_properties[price_columns] = df_properties[price_columns].astype('float64')

# Generate observation id
print("observation id")
df_properties['observation_id'] = df_properties.index.map(lambda x: str(uuid.uuid4()))

# Generate property_id
print("property id")
property_idetifiers = valuation_identifiers + ['property_account_id', 'latitude', 'latitude']
df_properties['property_id'] = df_properties[property_idetifiers].astype(str).apply(lambda x: ''.join(x), axis=1).map(lambda x: str(uuid.uuid5(uuid.NAMESPACE_DNS, x)))

datetime columns
id columns
valuation identifiers
price columns
observation id
property id


In [10]:
# see info
df_properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2339244 entries, 0 to 2339243
Data columns (total 60 columns):
 #   Column                                     Dtype         
---  ------                                     -----         
 0   half_bathrooms                             int64         
 1   full_bathrooms                             int64         
 2   property_class_more_common_in_area         int64         
 3   neighborhood                               object        
 4   constructor                                object        
 5   zip_code                                   int64         
 6   property_account_id                        object        
 7   special_common_installations_description   object        
 8   special_private_installations_description  object        
 9   complementary_common_works_description     object        
 10  age_in_months                              int64         
 11  parking_lots                               int64         
 12  

In [11]:
# count of valuation ids
df_properties[valuation_identifiers].nunique()

controller_id                1063
grantor_id                    141
valuation_unit_id             108
valuation_professional_id    2526
dtype: int64

In [12]:
# counts
for col in id_columns:
    print(df_properties[col].value_counts())

Series([], Name: count, dtype: int64)
potable_water_service_id
1    1541734
2       3842
3         19
0         17
Name: count, dtype: int64
lighting_service_id
3    908010
2    634422
1      3123
0        90
Name: count, dtype: int64
sidewalk_id
1    1468809
5      43483
4      14819
3       9555
2       1779
0        238
Name: count, dtype: int64
project_quality_id
1    1512106
3      10537
2       4685
0        144
4          1
5          1
Name: count, dtype: int64
property_class_id
3    1306412
4     835212
5     112339
2      59499
6      22474
7       3308
Name: count, dtype: int64
conservation_status_id
6    1126020
4    1062742
5     105379
7      40234
3       4869
Name: count, dtype: int64
elevator_service_id
2    1767828
0     417464
1      48142
Name: count, dtype: int64
state_id
19    268555
14    210831
15    196358
11    125855
9     120348
8     117914
5     112034
28    105086
30     97410
23     89251
22     84183
26     76312
25     74357
21     72178
13     66855
3

In [13]:
# see rows
df_properties

,half_bathrooms,full_bathrooms,property_class_more_common_in_area,neighborhood,constructor,zip_code,property_account_id,special_common_installations_description,special_private_installations_description,complementary_common_works_description,age_in_months,parking_lots,valuation_date,common_areas_completion_percentage,private_works_completion_percentage,potable_water_service_id,lighting_service_id,sidewalk_id,project_quality_id,property_class_id,conservation_status_id,controller_id,elevator_service_id,state_id,equipment_id,curb_id,infrastructure_service_id,road_materials_id,municipality_id,grantor_id,urban_proximity_id,water_collection_service_id,electrical_supply_service_id,telephone_service_supply_id,property_type_id,public_transportation_service_id,valuation_unit_id,valuation_professional_id,latitude,longitude,level,total_levels,purpose_of_appraisal,bedrooms,accessory_area,built_area,saleable_area,land_area,distance_to_public_transportation,rentable_units,rentable_units_subject_property,current_use,market_price,price,built_price,land_price,access_routes,remaining_useful_life,observation_id,property_id
0,1,2,4,VISTA HERMOSA HUITEPEC,NO APLICA,29263,<NA>,NaN,NaN,NaN,36,1,2022-06-14,NaN,100.0,<NA>,<NA>,<NA>,<NA>,4,4,301ef216-0137-4989-bdd0-32bf1adf1e9b,0,7,1,<NA>,1,2,78,6f8c15c7-2914-40bb-87ff-59076af288c9,3,<NA>,<NA>,<NA>,2,<NA>,35a7806a-f686-4159-be91-16997e86d271,87aa63b6-9005-4ed3-ba44-fcd75453a9e9,16.737283,-92.672659,3,3,S/D,3,10.75,216.81,227.56,302.70,NaN,1.0,NaN,NaN,3716737.0,3717000.0,2532281.13,1.033418e+06,NaN,67,02086af7-ffbb-4e67-b857-27e6cef09b0d,2b225ce9-7742-512e-aac5-846811f1a6a1
1,1,2,4,MAYA,NO APLICA,97134,<NA>,NaN,NaN,NaN,204,1,2022-11-15,NaN,100.0,<NA>,<NA>,<NA>,<NA>,4,4,717559f7-a918-48f7-9356-c79c582d95b3,0,31,2,<NA>,1,2,50,9a2781c9-b312-453c-86d1-a6e90584a083,2,<NA>,<NA>,<NA>,2,<NA>,b3bf58b2-40cc-4edd-bcf1-e4a0b2798533,02af4710-0922-4bed-9763-045ba14e0283,21.014297,-89.575189,3,2,S/D,3,13.38,141.55,154.93,198.00,NaN,1.0,NaN,NaN,2348706.0,2349000.0,1111845.21,9.848520e+05,NaN,53,43c375b2-75fd-4fcc-b311-55b396e7cbbf,b7f8c282-b67e-5d63-920d-c603b4e2eb52
2,0,1,3,KANASIN,NO APLICA,97370,<NA>,NaN,NaN,NaN,60,1,2022-06-22,NaN,100.0,<NA>,<NA>,<NA>,<NA>,3,5,8378ef1c-2fdf-4b2a-be29-0e62eb19d753,2,31,2,<NA>,1,3,41,98013144-6a9f-4e08-9255-cb54568d2f5a,2,<NA>,<NA>,<NA>,2,<NA>,4e63856b-ae68-4653-bcaa-86a51d5d7053,a7f6ef17-0933-4138-9aa9-99529929184d,20.928954,-89.546637,3,1,S/D,2,1.12,44.35,45.47,144.80,NaN,1.0,NaN,NaN,446378.0,450000.0,279611.30,1.026632e+05,NaN,55,e57ad294-8db8-4ca1-b01f-7ccbc5981922,552216f4-540b-500d-a4e7-6be55b2b6703
3,0,1,3,MULCHECHEN,NO APLICA,97370,<NA>,NaN,NaN,NaN,192,1,2022-11-01,NaN,100.0,<NA>,<NA>,<NA>,<NA>,3,4,8378ef1c-2fdf-4b2a-be29-0e62eb19d753,2,31,2,<NA>,1,3,41,98013144-6a9f-4e08-9255-cb54568d2f5a,3,<NA>,<NA>,<NA>,2,<NA>,4e63856b-ae68-4653-bcaa-86a51d5d7053,923b9bb1-9ab4-4984-93f1-c873b3c26977,20.916897,-89.580945,3,1,S/D,2,2.16,52.05,54.21,133.75,NaN,1.0,NaN,NaN,424468.0,424000.0,290492.55,9.683500e+04,NaN,44,f95c14fe-9acc-4b5c-872b-bd15d412a65d,7215360d-9d5e-538a-971f-15fbef435f54
4,0,3,4,POBLACION MOTUL,NO APLICA,97430,<NA>,NaN,NaN,NaN,372,0,2022-11-01,NaN,100.0,<NA>,<NA>,<NA>,<NA>,4,4,1f637d21-f0cb-43be-b989-5923e4c01d94,0,31,1,<NA>,1,2,52,bc0e00ba-8c35-465f-bbcd-61cedf83d6d8,1,<NA>,<NA>,<NA>,2,<NA>,90a0a9f7-fab3-4170-b3da-db49e122e41b,f6e764a4-141c-4b02-b902-4a158ec58c58,21.096391,-89.285366,3,2,S/D,2,19.05,229.93,248.98,844.52,NaN,1.0,NaN,NaN,2779000.0,2779000.0,1767043.79,6.418352e+05,NaN,39,74af41d9-b7b3-464d-9302-7fd1f231ee26,5b18adfe-606c-590f-bc76-d3eabfd0ba41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2339239,1,3,5,FRACC RES CAMPESTRE CONDADO SAYAVEDRA,NO APLICA,52938,<NA>,3,NaN,3,0,4,2019-06-05,NaN,NaN,<NA>,<NA>,<NA>,<NA>,5,6,6798e530-93e2-4060-abe0-d1121e934d3d,2,15,4,<NA>,

In [14]:
# memory usage in MB
df_properties.memory_usage().sum() / 1024**2

1070.821044921875

## Dtypes

Coerce to the smallest possible dtype

In [15]:
def safe_int_convert(series, dtype):
    try:
        return pd.to_numeric(series, errors='coerce').astype(dtype)
    except ValueError:
        return series

In [16]:
# map
dtypes_mapping = {
    'half_bathrooms': 'uint8',
    'full_bathrooms': 'uint8',
    'property_class_more_common_in_area': 'string',
    'neighborhood': 'string',
    'constructor': 'string',
    'zip_code': 'uint32',
    'property_account_id': 'string',
    'special_common_installations_description': 'string',
    'special_private_installations_description': 'string',
    'complementary_common_works_description': 'string',
    'age_in_months': 'uint8',
    'parking_lots': 'uint8',
    'valuation_date': 'datetime64[ns]',
    'common_areas_completion_percentage': 'float32',
    'private_works_completion_percentage': 'float32',
    'potable_water_service_id': 'uint8',
    'lighting_service_id': 'uint8',
    'sidewalk_id': 'uint8',
    'project_quality_id': 'uint8',
    'property_class_id': 'uint8',
    'conservation_status_id': 'uint8',
    'controller_id': 'string',
    'elevator_service_id': 'uint8',
    'state_id': 'uint8',
    'equipment_id': 'uint8',
    'curb_id': 'uint8',
    'infrastructure_service_id': 'uint8',
    'road_materials_id': 'uint8',
    'municipality_id': 'uint16',
    'grantor_id': 'string',
    'urban_proximity_id': 'uint8',
    'water_collection_service_id': 'uint8',
    'electrical_supply_service_id': 'uint8',
    'telephone_service_supply_id': 'uint8',
    'property_type_id': 'uint8',
    'public_transportation_service_id': 'uint8',
    'valuation_unit_id': 'string',
    'valuation_professional_id': 'string',
    'latitudee': 'float64',
    'latitudeee': 'float64',
    'level': 'uint8',
    'total_levels': 'uint8',
    'purpose_of_appraisal': 'string',
    'bedrooms': 'uint8',
    'accessory_area': 'float32',
    'built_area': 'float32',
    'saleable_area': 'float32',
    'land_area': 'float32',
    'distance_to_public_transportation': 'float32',
    'rentable_units': 'uint32',
    'rentable_units_subject_property': 'uint32',
    'current_use': 'string',
    'market_price': 'float64',
    'price': 'float64',
    'built_price': 'float64',
    'land_price': 'float64',
    'access_routes': 'string',
    'remaining_useful_life': 'int16',  # some errors on this column
    'observation_id': 'string',
    'property_id': 'string'
}

# Convert
# all int columns first nan to np.nan
print("dtype int columns")
# any int column
int_columns = [col for col in dtypes_mapping if re.search('int', dtypes_mapping[col])]
# int columns
for col in int_columns:
    df_properties[col] = safe_int_convert(df_properties[col], dtypes_mapping[col])


# convert other columns
print("dtype other columns")
dtypes_non_integer = {k: v for k, v in dtypes_mapping.items() if k not in int_columns}
df_properties = df_properties.astype(dtypes_non_integer)

dtype int columns
dtype other columns


In [17]:
# see info
df_properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2339244 entries, 0 to 2339243
Data columns (total 60 columns):
 #   Column                                     Dtype         
---  ------                                     -----         
 0   half_bathrooms                             uint8         
 1   full_bathrooms                             uint8         
 2   property_class_more_common_in_area         string        
 3   neighborhood                               string        
 4   constructor                                string        
 5   zip_code                                   uint32        
 6   property_account_id                        string        
 7   special_common_installations_description   string        
 8   special_private_installations_description  string        
 9   complementary_common_works_description     string        
 10  age_in_months                              uint8         
 11  parking_lots                               uint8         
 12  

In [18]:
# memory usage in MB
df_properties.memory_usage().sum() / 1024**2

738.4203834533691

In [19]:
# counts
for col in id_columns:
    print(df_properties[col].value_counts())

Series([], Name: count, dtype: Int64)
potable_water_service_id
1    1541734
2       3842
3         19
0         17
Name: count, dtype: int64
lighting_service_id
3    908010
2    634422
1      3123
0        90
Name: count, dtype: int64
sidewalk_id
1    1468809
5      43483
4      14819
3       9555
2       1779
0        238
Name: count, dtype: int64
project_quality_id
1    1512106
3      10537
2       4685
0        144
4          1
5          1
Name: count, dtype: int64
property_class_id
3    1306412
4     835212
5     112339
2      59499
6      22474
7       3308
Name: count, dtype: int64
conservation_status_id
6    1126020
4    1062742
5     105379
7      40234
3       4869
Name: count, dtype: int64
elevator_service_id
2    1767828
0     417464
1      48142
Name: count, dtype: int64
state_id
19    268555
14    210831
15    196358
11    125855
9     120348
8     117914
5     112034
28    105086
30     97410
23     89251
22     84183
26     76312
25     74357
21     72178
13     66855
3

---
# Save

In [20]:
# save
print("saving ...")
df_properties.to_parquet(folder_data + 'appraisals.parquet', index=False)    

saving ...


---
# Sandbox

In [21]:
# count of valuation ids
df_properties[valuation_identifiers].nunique()

controller_id                1063
grantor_id                    141
valuation_unit_id             108
valuation_professional_id    2526
dtype: int64

In [22]:
# count of property ids
df_properties['property_id'].value_counts().sort_values(ascending=False)

property_id
e53135c9-dcf8-5cd2-9b66-689e536ab6c3    319
18e27699-a0d1-5ecb-88ca-35b4d6bae7d8    239
77b3182c-edd9-5b1b-93dc-bf96f7c4fd3b    202
365c430e-13f8-51d7-b964-bd10723edf86    188
0fa57e0a-a526-563d-aab8-9f7b2acc3768    141
                                       ... 
b626ad33-5f49-5255-9e2c-5a87b1be0c80      1
85ff3227-a44d-58a6-9045-f97a5e464910      1
3e31b5d6-1789-5d2f-9d5d-1bcac2dfc8f9      1
49a32f6f-2fb9-58bf-82e7-131fb88a1bfb      1
2140f4ef-c15b-50e7-9ab1-f5017285d63c      1
Name: count, Length: 2093312, dtype: Int64

In [23]:
# count of repeated properties (more than one)
id_repetead_properties = df_properties['property_id'].value_counts()[df_properties['property_id'].value_counts().gt(1)].index

# count of repeated properties
print(f"repeated properties: {len(id_repetead_properties)}")
print(f"total obs: {df_properties['property_id'].value_counts()[id_repetead_properties].sum()}")
print(f"avg repeated properties: {df_properties['property_id'].value_counts()[id_repetead_properties].mean()}")

repeated properties: 147400
total obs: 393332
avg repeated properties: 2.6684667571234737


In [24]:
# count of observation ids
df_properties['remaining_useful_life'].describe()

count    2.339244e+06
mean     5.847166e+01
std      1.112246e+01
min     -9.000000e+00
25%      5.200000e+01
50%      6.000000e+01
75%      6.900000e+01
max      8.000000e+01
Name: remaining_useful_life, dtype: float64

In [25]:
# describe id columns
df_properties[id_columns].describe()

,property_class_id,conservation_status_id,state_id,equipment_id,infrastructure_service_id,road_materials_id,municipality_id,urban_proximity_id,property_type_id
count,2.339244e+06,2.339244e+06,2.339244e+06,2.339244e+06,2.339244e+06,2.339244e+06,2.339244e+06,2.339244e+06,2.339244e+06
mean,3.462134e+00,5.057287e+00,1.666580e+01,3.349917e+00,3.041614e+00,2.788298e+00,4.028076e+01,2.553759e+00,2.769766e+00
std,6.856368e-01,1.004745e+00,7.953309e+00,8.482336e-01,3.551336e-01,1.313538e+00,4.494025e+01,7.505294e-01,8.627087e-01
min,2.000000e+00,3.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00
25%,3.000000e+00,4.000000e+00,1.100000e+01,3.000000e+00,3.000000e+00,2.000000e+00,1.100000e+01,2.000000e+00,2.000000e+00
50%,3.000000e+00,5.000000e+00,1.500000e+01,4.000000e+00,3.000000e+00,2.000000e+00,2.700000e+01,3.000000e+00,3.000000e+00
75%,4.000000e+00,6.000000e+00,2.300000e+01,4.000000e+00,3.000000e+00,3.000000e+00,5.000000e+01,3.000000e+00,3.000000e+00
max,7.000000e+00,7.000000e+00,3.200000e+01,4.000000e+00,4.000000e+00,8.000000e+00,5.700000e+02,5.000000e+00,5.000000e+00


In [26]:
# municipality
df_properties['state_id']

0           7
1          31
2          31
3          31
4          31
           ..
2339239    15
2339240     5
2339241    19
2339242    19
2339243    24
Name: state_id, Length: 2339244, dtype: uint8

In [27]:
# describe price
cols = [
    'remaining_useful_life'
]
df_properties[cols].describe()

,remaining_useful_life
count,2.339244e+06
mean,5.847166e+01
std,1.112246e+01
min,-9.000000e+00
25%,5.200000e+01
50%,6.000000e+01
75%,6.900000e+01
max,8.000000e+01


In [28]:
# read parquet
df2 = pd.read_parquet(folder_data + 'appraisals.parquet')

# see info
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2339244 entries, 0 to 2339243
Data columns (total 60 columns):
 #   Column                                     Dtype         
---  ------                                     -----         
 0   half_bathrooms                             uint8         
 1   full_bathrooms                             uint8         
 2   property_class_more_common_in_area         string        
 3   neighborhood                               string        
 4   constructor                                string        
 5   zip_code                                   uint32        
 6   property_account_id                        string        
 7   special_common_installations_description   string        
 8   special_private_installations_description  string        
 9   complementary_common_works_description     string        
 10  age_in_months                              uint8         
 11  parking_lots                               uint8         
 12  